In [24]:
import math
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pymrm as mrm
from IPython.display import display, clear_output

In [25]:
def holdup_correlation(U, rho_g, eps_s, D_T):
    eps_df_ref  = 0.27
    V_small_ref = 0.095
    rho_g_ref = 1.3
    eps_trans = np.power(rho_g/rho_g_ref,0.48)*(eps_df_ref - 0.7*eps_s)
    V_small = V_small_ref + 0.8*eps_s
    U_df = np.minimum(U,eps_trans * V_small)
    eps_df = U_df/V_small
    U_b = U - U_df
    eps_b = 0.3*np.power(D_T,-0.18)*np.power(U_b,0.58)
    return eps_b, U_b, eps_df, U_df 

def kla_correlation(eps_b, eps_df):
    D_L = np.array([[45.5E-9, 17.2E-9]])
    D_L_ref = 2e-9
    kla = np.sqrt(D_L/D_L_ref)*np.array([[0.5],[1.0]])*np.array([[eps_b],[eps_df]])
    return kla

solubilities = [[2.478, 2.964]]

U = 0.3
eps_b, U_b, eps_df, U_df = holdup_correlation(U, 7.0, 0.3, 7.0)
kla =  kla_correlation(eps_b, U_b)

print(eps_b, U_b, eps_df, U_df)
print(kla)
#plt.plot(U, eps_b + (1-eps_b)*eps_df)
#plt.show()

0.09565348645821949 0.2549027242662663 0.13461873353353343 0.04509727573373369
[[0.22811903 0.14025554]
 [1.21580851 0.74752152]]


In [26]:
class SlurryBubbleColumn:
    def __init__(self):
        self.num_c = 2  # components: 0: CO, 1: H2
        self.num_ph = 3 # phases: 0: b, 1: df, 2: sl
        self.num_x = 100
        
        D_L = np.array([[[45.5E-9, 17.2E-9]]])
        D_Lref = 2e-9
        self.kla = np.sqrt(D_L/D_Lref)*np.array([[0.5,1.0]])
        
        self.k1 = 10
        self.k2 = 0
        self.dt = 1e-2
        self.L = 1
        self.maxfev = 2
        self.plot_interval = 10
        self.plot_cntr = 0
        self.c_in = 10
        self.v = [[1,-1, 1, -1]]
        self.bc = {'a': [[[0,1,0,1]], [[1, 0,1,0]]], 'b': [[[1,0,1,0]], [[0, 1,0,1]]], 'd':[ [[self.c_in, 0, 0, 0]], [[0, self.c_in, 0, 0]]]}
        self.x_f = np.linspace(0, self.L, self.num_x+1)
        #self.x_f = mrm.non_uniform_grid(0, L, num_x+1, 0.05*L, 0.75)
        self.x_c = 0.5*(self.x_f[:-1] + self.x_f[1:])
        Conv, conv_bc = mrm.construct_convflux_upwind((self.num_x,self.num_c), self.x_f, self.x_c, self.bc, self.v, axis=0)
        self.Div = mrm.construct_div((self.num_x,self.num_c), self.x_f, nu=0, axis=0)
        self.g_const = self.Div @ conv_bc.reshape((-1,1))
        self.Jac_const = sp.sparse.eye_array(self.num_x*self.num_c, format='csc')/self.dt + self.Div @ Conv
        self.Jac_conv = self.Div @ Conv
        self.init_c()
    
    def 
        
    def init_c(self):
        self.c = np.zeros((self.num_x, self.num_c))
        self.fig, self.ax = plt.subplots()
        self.ax.set_xlabel('Position')
        self.ax.set_ylabel('Concentration')
        
    def reaction(self, c):
        r = self.k1*self.c[:,0]*self.c[:,1]- self.k2*self.c[:,2]*self.c[:,3]
        f = np.zeros_like(self.c)
        f[:,0] = -r
        f[:,1] = -r
        f[:,2] = r
        f[:,3] = r
        return f

    def g(self, c):
        g_react, Jac_react = mrm.numjac_local(self.reaction, c, axis=-1)
        c_f, dc_f = mrm.interp_cntr_to_stagg_tvd(c, self.x_f, self.x_c, self.bc, self.v, mrm.minmod, axis=0)
        g_conv = self.Div @ (self.v*c_f).reshape((-1,1))
        g = (c-self.c_old).reshape((-1,1))/self.dt + g_conv - g_react.reshape((-1,1))
        Jac = self.Jac_const - Jac_react
        return g, Jac

    def plot(self):
        if (self.plot_cntr == 0):
            clear_output(wait=True)
            for i in range(self.num_c):
                self.ax.plot(self.x_c, self.c[:, i])
            display(self.fig)
            plt.show(self.fig)
        self.plot_cntr = (self.plot_cntr + 1) % self.plot_interval

    def solve(self, num_time_steps):
        for i in range(num_time_steps):
            self.c_old = self.c.copy()
            result = mrm.newton(self.g, self.c, maxfev = self.maxfev, callback = lambda x,g: mrm.clip_approach(x, g, upper_bounds = self.c_in, factor = 0.5))
            self.c = result.x
            self.plot()

SyntaxError: invalid syntax (3662294910.py, line 31)